In [ ]:
## 26.08.2025
## 1- popular-based recommendation system
## 2- content-base recommendation
## 3-collabrative-based recommendation

# Import Libraries

In [2]:
import pandas as pd
import numpy as np


In [3]:
from google.colab import  drive,files
drive.mount("/content/drive")
files.upload()

Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"oula23","key":"23097febc5c9b1ac6a4b70d11ff71f01"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download samlearner/letterboxd-movie-ratings-data


Dataset URL: https://www.kaggle.com/datasets/samlearner/letterboxd-movie-ratings-data
License(s): Attribution 4.0 International (CC BY 4.0)
 67% 126M/188M [00:00<00:00, 1.31GB/s]
100% 188M/188M [00:00<00:00, 903MB/s] 


In [6]:
import zipfile

with zipfile.ZipFile('/content/letterboxd-movie-ratings-data.zip', 'r') as zip_ref:
    zip_ref.extractall('/kaggle/temp')

# Movie Dataset

In [7]:
movie_dataset=pd.read_csv("/kaggle/temp/movie_data.csv",engine="python")

In [ ]:
movie_dataset.shape

(285963, 19)

In [ ]:
movie_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285963 entries, 0 to 285962
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _id                   285963 non-null  object 
 1   genres                274986 non-null  object 
 2   image_url             264521 non-null  object 
 3   imdb_id               243898 non-null  object 
 4   imdb_link             243914 non-null  object 
 5   movie_id              285616 non-null  object 
 6   movie_title           283453 non-null  object 
 7   original_language     274986 non-null  object 
 8   overview              261362 non-null  object 
 9   popularity            274872 non-null  float64
 10  production_countries  274854 non-null  object 
 11  release_date          270954 non-null  object 
 12  runtime               269943 non-null  float64
 13  spoken_languages      274758 non-null  object 
 14  tmdb_id               279803 non-null  float64
 15  

In [8]:
# remove unwanted columns
movie_dataset.drop(["imdb_link","image_url","tmdb_link"],inplace=True,axis=1)

In [ ]:
movie_dataset["overview"][10]

"Hookers are being killed all over the place, their bodies dumped in various parts of the city. Now, it's up to Officer Kelly Anderson with the help of Detective Grant Jordan to go undercover as a hooker to stop these horrific killings."

In [ ]:
movie_dataset.head(5)

,_id,genres,imdb_id,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,tmdb_id,vote_average,vote_count,year_released
0,5fc85f606758f69634496fd3,"[""Music"",""Animation""]",NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],535272.0,0.0,0.0,1971.0
1,5fc85ff26758f696344ace0c,[],tt0586129,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],318331.0,8.0,1.0,1960.0
2,5fc85f606758f69634496fcd,"[""Drama""]",tt0045731,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",117779.0,6.6,10.0,1953.0
3,5fc85f606758f69634496fd1,"[""Drama""]",tt0187327,the-musicians-daughter,The Musician's Daughter,en,Carl Wagner's good wife was dying. His heart b...,0.600,"[""United States of America""]",1911-12-12,15.0,[],560377.0,0.0,0.0,1911.0
4,5fc85f606758f69634496fd4,"[""Documentary""]",tt4769914,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],525187.0,0.0,0.0,2018.0


# Ratings Dataset

In [9]:
ratings_dataset=pd.read_csv("/kaggle/temp/ratings_export.csv")
ratings_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11078167 entries, 0 to 11078166
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   _id         object
 1   movie_id    object
 2   rating_val  int64 
 3   user_id     object
dtypes: int64(1), object(3)
memory usage: 338.1+ MB


In [ ]:
ratings_dataset

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof
...,...,...,...,...
11078162,6239f4f1a936b95600b3d798,alien-3,6,turnitip
11078163,6239f4f1a936b95600b3d799,battleship-potemkin,7,turnitip
11078164,6239f4f1a936b95600b3d79e,pusher,6,turnitip
11078165,6239f4f1a936b95600b3d7a1,wild-strawberries,7,turnitip


In [ ]:
ratings_dataset["movie_id"]=="gainsbourg-toute-une-vi"

,movie_id
0,False
1,False
2,False
3,False
4,False
...,...
11078162,False
11078163,False
11078164,False
11078165,False


In [10]:

## take random samples from datasets
import random
SAMPLE_SIZE=200000
random.seed(72)
indices = random.sample(range(len(movie_dataset)), SAMPLE_SIZE)
indices.sort()
movie_dataset=movie_dataset.loc[indices]

In [11]:
ratings_dataset=ratings_dataset.loc[indices]

In [ ]:
ratings_dataset.shape

(200000, 4)

# Users Datasets

In [ ]:
users_dataset=pd.read_csv("/kaggle/temp/users_export.csv")
users_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8139 entries, 0 to 8138
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                8139 non-null   object 
 1   display_name       7832 non-null   object 
 2   num_ratings_pages  4747 non-null   float64
 3   num_reviews        8139 non-null   int64  
 4   username           8139 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 318.1+ KB


In [ ]:
users_dataset.head(5)

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof
1,5fc4172ec6cd28ebd99dd0ea,Matt Singer,52.0,1915,superpulse
2,5fc4172ec6cd28ebd99dd0ed,Sean Baker,21.0,1283,lilfilm
3,5fc4172ec6cd28ebd99dd0ee,iana,37.0,1177,ianamurray
4,5fc419171ebf67b9fbe48615,Lizzy,57.0,1810,punchdrunklizzy


In [ ]:
users_dataset[users_dataset["username"]=="deathproof"]

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof


# popularity-based recommendation

In [ ]:
popular_movie_dataset=movie_dataset[(movie_dataset["vote_count"]>100)&(movie_dataset["vote_average"]>6)]

In [ ]:
popular_movie_dataset.shape

(10675, 16)

In [ ]:
# the most popular movie names
popular_movie_dataset=popular_movie_dataset[['movie_title',"vote_average","vote_count","movie_id"]].sort_values(by="vote_count",ascending=False)

In [ ]:
popular_movie_dataset.shape

(10675, 4)

In [ ]:
popular_movie_dataset.head(5)

,movie_title,vote_average,vote_count,movie_id
42305,Inception,8.3,30211.0,inception
40625,Interstellar,8.3,27020.0,interstellar
82155,The Dark Knight,8.5,26173.0,the-dark-knight
18551,Deadpool,7.6,25753.0,deadpool
64959,The Avengers,7.7,25672.0,the-avengers-2012


In [ ]:
## the most popular movies that the user hasnt seen

seen_movie_ids=ratings_dataset[ratings_dataset["user_id"]=="adrianbalboa"]["movie_id"]

def get_unseen_popular_movies(user_id,popular_movie_dataset):
  seen_movie_list=ratings_dataset[ratings_dataset["user_id"]==user_id]["movie_id"].values.tolist()
  popular_seen_movies=popular_movie_dataset[popular_movie_dataset["movie_id"].isin(seen_movie_list)]
  popular_unseen_movies=popular_movie_dataset.drop(popular_seen_movies.index,axis=0)
  print("Most popular unseen Movies -User Specific-")
  return popular_unseen_movies["movie_id"].values.tolist()[0:5]

def get_all_users_unseen_popular_movies():
  pass


# Weighted Average

In [ ]:
v=movie_dataset['vote_count']
R=movie_dataset["vote_average"]
C=movie_dataset["vote_average"].mean()
m=movie_dataset["vote_count"].quantile(0.70)


In [ ]:
movie_dataset["weighted_average"]=((R*v)+(C*m))/(v+m)

In [ ]:
ranking=movie_dataset.sort_values(by="weighted_average",ascending=False)
print("Top movies by Weighted Average :\n")
ranking[["movie_title","weighted_average"]].iloc[0:5]

Top movies by Weighted Average :



,movie_title,weighted_average
106249,Bring the Soul: The Movie,9.196789
198291,Burn the Stage: The Movie,9.103281
193354,Break the Silence: The Movie,8.975981
99044,Franco Escamilla: Por La Anécdota,8.875887
226315,Franco Escamilla: Por La Anécdota,8.875887


# Content-Based Filtering

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

vectorizer=TfidfVectorizer(
    analyzer="word",
    ngram_range=(1,2),
    min_df=0.003,
    max_df=0.5,
    max_features=5000,
    stop_words=stopwords.words("english")
)




In [14]:
movies_ids=ratings_dataset["movie_id"].tolist()

In [15]:
tfidf_matrix=vectorizer.fit_transform(movie_dataset["overview"].values.astype('U'))

In [16]:
tfidf_feature_names =vectorizer.get_feature_names_out()

In [ ]:
tfidf_matrix.shape

(200000, 1443)

create item profile and user profile

In [17]:
from sklearn.model_selection import train_test_split

movie_rating_train_df, movie_rating_test_df=train_test_split(ratings_dataset,test_size=0.2)



In [ ]:
movies_ids.index("miss-you-already")

5

In [18]:
import scipy
from tqdm import tqdm
from  sklearn.preprocessing import normalize

def get_movie_profile(movie_id):
  idx=movies_ids.index(movie_id)
  movie_profile=tfidf_matrix[idx:idx+1]
  if movie_profile.shape==(0,1436):
    M = scipy.sparse.csr_matrix((1, 1436))
    return M
  return movie_profile


def get_movie_profiles(movie_ids):
  print("\n create all movies profiles \n")
  movie_profiles_list=[get_movie_profile(x) for x in tqdm(movie_ids)]
  movie_profiles=scipy.sparse.vstack(movie_profiles_list)
  print(f"Item profile list LENGTH : {len(movie_profiles_list)} ")
  print(f"movies profiles SHAPE : {movie_profiles.shape} ")

  return movie_profiles

def get_user_profile(user_id,rating_dataset):
  ratings_user_df=rating_dataset.loc[user_id]
  user_movie_profile=get_movie_profiles(ratings_user_df["movie_id"])
  user_ratings_strength=np.array(ratings_user_df["rating_val"]).reshape(-1,1)
  user_movie_weighted_avg= np.sum(user_movie_profile.multiply(user_ratings_strength),axis=0)/np.sum(user_ratings_strength)
  user_movie_weighted_avg = np.asarray(user_movie_weighted_avg).ravel()
  user_weighted_avg_norm=normalize([user_movie_weighted_avg])
  return user_weighted_avg_norm

def get_users_profiles():
  ratings_indexed_df=movie_rating_train_df[movie_rating_train_df["movie_id"].isin(ratings_dataset["movie_id"])].set_index("user_id")
  users_profile={}
  print("Create all users profiles \n")
  for user_id in tqdm(ratings_indexed_df.index.unique()):
    print(f"user id {user_id}")
    users_profile[user_id]=get_user_profile(user_id,ratings_indexed_df)
  return users_profile


'embers-2015'

In [19]:
users_profiles=get_users_profiles()


Create all users profiles 



  0%|          | 0/168 [00:00<?, ?it/s]

user id peterstanley

 create all movies profiles 




  1%|          | 1/168 [00:00<02:37,  1.06it/s]

Item profile list LENGTH : 1314 
movies profiles SHAPE : (1314, 1443) 
user id adashelby

 create all movies profiles 



...

Item profile list LENGTH : 267 
movies profiles SHAPE : (267, 1443) 
user id ellefnning

 create all movies profiles 




 99%|█████████▉| 167/168 [01:33<00:00,  6.84it/s]

Item profile list LENGTH : 329 
movies profiles SHAPE : (329, 1443) 
user id reygf

 create all movies profiles 




100%|██████████| 168/168 [01:33<00:00,  1.79it/s]

Item profile list LENGTH : 451 
movies profiles SHAPE : (451, 1443) 


In [20]:
my_profile=users_profiles["cevangelista413"]
print(my_profile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names,users_profiles["cevangelista413"].flatten().tolist()),key=lambda x:-x[1])[:20],columns=["token","relevance"])

(1, 1443)


,token,relevance
0,nan,0.457150
1,film,0.206631
2,life,0.155590
3,man,0.137663
4,one,0.132903
5,young,0.129703
6,love,0.126259
7,new,0.113064
8,story,0.101445
9,girl,0.098835


In [ ]:
users_profiles["deathproof"].shape

(1, 1443)

In [70]:
from sklearn.metrics.pairwise import cosine_similarity
## content-based model

class ContentBasedModel:
  MODEL_NAME="Content-Based"

  def __init__(self,movies_df=None):
    self.movies_ids=movies_ids
    self.movies_df=movies_df

    # ,tfidf_matrix=None,users_profiles=Non
    # self.tfidf_matrix=tfidf_matrix,
    # self.users_profiles=users_profiles
  def get_model_name(self):
    return self.MODEL_NAME

  def __get_recommended_movies(self,user_id,topn=1000):
    similarity=cosine_similarity(users_profiles[user_id],tfidf_matrix)
    similar_indices=similarity.argsort().flatten()[-1000:]
    similar_movies=sorted([(movies_ids[i],similarity[0,i]) for i in similar_indices],key=lambda x:-x[1])
    return similar_movies

  def recommended_movies(self,user_id,movies_to_ignore=[],topn=10,verbose=False):
    similar_movies=self.__get_recommended_movies(user_id)
    similar_movies_filtered=list(filter(lambda x:x[0] not in movies_to_ignore,similar_movies))
    recommendation_df=pd.DataFrame(similar_movies_filtered,columns=["movie_id","recStrength"]) \
    .head(topn)

    if verbose:
      if self.movies_df is None:
        raise Exception("movies_df is required in verbose mode")
      recommendation_df=recommendation_df.merge(self.movies_df,how="left",
                                                left_on="movie_id",
                                                right_on="movie_id")[["recStrength","movie_id","movie_title"]]
    return recommendation_df


In [71]:
content_based_recommender_model = ContentBasedModel(movie_dataset)


In [77]:

content_based_recommender_model.recommended_movies("superpulse", movies_to_ignore=[1], topn=5, verbose=True)

,recStrength,movie_id,movie_title
0,0.43523,blue-valentine,Blue Valentine
1,0.43523,when-inge-is-dancing,When Inge Is Dancing
2,0.43523,sin-city,Sin City
3,0.43523,cinema-paradiso,Cinema Paradiso
4,0.43523,taxi-driver,Taxi Driver


In [ ]:
# __get_recommended_items("deathproof",movies_ids)

[('see-you-up-there', np.float64(0.5907229340644958)),
 ('brothers-a-family-film', np.float64(0.5907229340644958)),
 ('hollywood-boulevard', np.float64(0.5907229340644958)),
 ('the-burbs', np.float64(0.5907229340644958)),
 ('missing-in-action', np.float64(0.5907229340644958)),
 ('dont-answer-the-door', np.float64(0.5907229340644958)),
 ('rocky-balboa', np.float64(0.5907229340644958)),
 ('godzilla-2014', np.float64(0.5907229340644958)),
 ('safety-not-guaranteed', np.float64(0.5907229340644958)),
 ('the-way-he-looks', np.float64(0.5907229340644958)),
 ('finding-dory', np.float64(0.5907229340644958)),
 ('brick', np.float64(0.5907229340644958)),
 ('raging-bull', np.float64(0.5907229340644958)),
 ('amores-perros', np.float64(0.5907229340644958)),
 ('right-now-wrong-then', np.float64(0.5907229340644958)),
 ('nerve-2016', np.float64(0.5907229340644958)),
 ('legend-2015', np.float64(0.5907229340644958)),
 ('let-the-right-one-in', np.float64(0.5907229340644958)),
 ('titanic-1997', np.float64(0.